<a href="https://colab.research.google.com/github/akmaldika/Task-IF5153-NLP/blob/main/13521070_praktikum3_a.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Praktikum 3
> Text Classification a

NIM: 13521070<br>
Nama: Akmal Mahardika Nurwahyu Pratama

## Lab Requirements

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
%pip install datasets tdqm transformers torch

  Preparing metadata (setup.py) ... done
INFO: pip is looking at multiple versions of multiprocess to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 471.6/471.6 kB 7.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 8.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 5.6 MB/s eta 0:00:00
  Created wheel for tdqm: filename=tdqm-0.0.1-py3-none-any.whl size=1321 sha256=d39c59d24457403483fff9b216a5aef671fe91a9cac96d5179855c2685941f14
  Stored in directory: /root/.cache/pip/wheels/37/31/b8/7b711038035720ba0df14376af06e5e76b9bd61759c861ad92
Successfully built tdqm


In [3]:
class Config:
  BASE_PATH = "/content/drive/MyDrive/ITB/Semester 7/IF5153 Pemrosesan Bahasa Alami/Praktikum3"

Config = Config()

## Text Clasification 2a

### Dataset

In [4]:
from datasets import load_dataset

Config.DATASET_NAME = "rotten_tomatoes"

dataset = load_dataset(Config.DATASET_NAME)

dataset

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/7.46k [00:00<?, ?B/s]

train.parquet:   0%|          | 0.00/699k [00:00<?, ?B/s]

validation.parquet:   0%|          | 0.00/90.0k [00:00<?, ?B/s]

test.parquet:   0%|          | 0.00/92.2k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/8530 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/1066 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1066 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 8530
    })
    validation: Dataset({
        features: ['text', 'label'],
        num_rows: 1066
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 1066
    })
})

### Model

In [5]:
Config.MODEL_NAME = "distilbert/distilbert-base-uncased"
Config.MODEL_PATH = f"{Config.BASE_PATH}/models/{Config.MODEL_NAME}"
Config.TOKENIZER_PATH = f"{Config.BASE_PATH}/models/{Config.MODEL_NAME}_tokenizer"

In [6]:
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import os

device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
if os.path.exists(Config.MODEL_PATH) and os.path.exists(Config.TOKENIZER_PATH):
  classification_model = AutoModelForSequenceClassification.from_pretrained(Config.MODEL_PATH, num_labels=2)
  tokenizer = AutoTokenizer.from_pretrained(Config.TOKENIZER_PATH)
else:
  classification_model = AutoModelForSequenceClassification.from_pretrained(Config.MODEL_NAME, num_labels=2)
  tokenizer = AutoTokenizer.from_pretrained(Config.MODEL_NAME)

  classification_model.save_pretrained(Config.MODEL_PATH)
  tokenizer.save_pretrained(Config.TOKENIZER_PATH)

classification_model.to(device)

The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.


0it [00:00, ?it/s]

DistilBertForSequenceClassification(
  (distilbert): DistilBertModel(
    (embeddings): Embeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (transformer): Transformer(
      (layer): ModuleList(
        (0-5): 6 x TransformerBlock(
          (attention): MultiHeadSelfAttention(
            (dropout): Dropout(p=0.1, inplace=False)
            (q_lin): Linear(in_features=768, out_features=768, bias=True)
            (k_lin): Linear(in_features=768, out_features=768, bias=True)
            (v_lin): Linear(in_features=768, out_features=768, bias=True)
            (out_lin): Linear(in_features=768, out_features=768, bias=True)
          )
          (sa_layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (ffn): FFN(
            (dropout): Dropout(p=0.1, inplace=False)
 

### Train

In [7]:
Config.TOKENIZED_DATASET_PATH = f"{Config.BASE_PATH}/tokenized_datasets"

In [8]:
from transformers import BatchEncoding, DataCollatorWithPadding
from datasets import Dataset, load_from_disk

def tokenize_function(isinstances_text: Dataset) -> BatchEncoding:
    return tokenizer(isinstances_text["text"],
                     padding="max_length",
                     truncation=True)

if os.path.exists(Config.TOKENIZED_DATASET_PATH):
  tokenized_datasets = load_from_disk(Config.TOKENIZED_DATASET_PATH)
else:
  tokenized_datasets = dataset.map(tokenize_function)
  tokenized_datasets = tokenized_datasets.rename_column("label", "labels")
  tokenized_datasets.set_format(type="torch", columns=["input_ids", "attention_mask", "labels"])
  tokenized_datasets.save_to_disk(Config.TOKENIZED_DATASET_PATH)

data_collator = DataCollatorWithPadding(tokenizer=tokenizer)
display(data_collator)
display(tokenized_datasets)

DataCollatorWithPadding(tokenizer=DistilBertTokenizerFast(name_or_path='/content/drive/MyDrive/ITB/Semester 7/IF5153 Pemrosesan Bahasa Alami/Praktikum3/models/distilbert/distilbert-base-uncased_tokenizer', vocab_size=30522, model_max_length=512, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'unk_token': '[UNK]', 'sep_token': '[SEP]', 'pad_token': '[PAD]', 'cls_token': '[CLS]', 'mask_token': '[MASK]'}, clean_up_tokenization_spaces=True),  added_tokens_decoder={
	0: AddedToken("[PAD]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	100: AddedToken("[UNK]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	101: AddedToken("[CLS]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	102: AddedToken("[SEP]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	103: AddedToken("[MASK]", rstrip=False, lstrip=False, single_word=False, normaliz

DatasetDict({
    train: Dataset({
        features: ['text', 'labels', 'input_ids', 'attention_mask'],
        num_rows: 8530
    })
    validation: Dataset({
        features: ['text', 'labels', 'input_ids', 'attention_mask'],
        num_rows: 1066
    })
    test: Dataset({
        features: ['text', 'labels', 'input_ids', 'attention_mask'],
        num_rows: 1066
    })
})

In [9]:
Config.BATCH_SIZE = 8
Config.NUM_EPOCHS = 1
Config.TUNNED_PATH = f"{Config.BASE_PATH}/tuned_models/{Config.MODEL_NAME}"
Config.RESULT_PATH = f"{Config.BASE_PATH}/results"

In [11]:
from transformers import Trainer, TrainingArguments
from sklearn.metrics import accuracy_score

def compute_metrics(pred):
  labels = pred.label_ids
  preds = pred.predictions.argmax(-1)
  accuracy = accuracy_score(labels, preds)
  return {'accuracy': accuracy}

training_args = TrainingArguments(
    output_dir=Config.RESULT_PATH,
    num_train_epochs=Config.NUM_EPOCHS,
    per_device_train_batch_size=Config.BATCH_SIZE,
    per_device_eval_batch_size=Config.BATCH_SIZE,

    eval_strategy="epoch",
    fp16=True,
    )

if os.path.exists(Config.TUNNED_PATH):
  classification_model = AutoModelForSequenceClassification.from_pretrained(Config.TUNNED_PATH, num_labels=2)
  trainer = Trainer(
      model=classification_model,
      args=training_args,
      train_dataset=tokenized_datasets["train"],
      eval_dataset=tokenized_datasets["validation"],
      compute_metrics=compute_metrics
      )
else:

  trainer = Trainer(
      model=classification_model,
      args=training_args,
      train_dataset=tokenized_datasets["train"],
      eval_dataset=tokenized_datasets["validation"],
      compute_metrics=compute_metrics
      )

  trainer.train()
  trainer.save_model(Config.TUNNED_PATH)

### Validation

In [14]:
Config.EVALUATION_PATH = f"{Config.BASE_PATH}/evaluation"

In [19]:
# Evaluate on validation set
# trainer.evaluate(tokenized_datasets["validation"])
import json

if os.path.exists(Config.EVALUATION_PATH):
  with open(f"{Config.EVALUATION_PATH}/eval_results.json", "r") as f:
    evaluation = json.load(f)
else:
  evaluation = trainer.evaluate(tokenized_datasets["validation"])
  os.makedirs(Config.EVALUATION_PATH, exist_ok=True)
  with open(f"{Config.EVALUATION_PATH}/eval_results.json", "w") as f:
    json.dump(evaluation, f)

display(evaluation)


{'eval_loss': 0.35670530796051025,
 'eval_model_preparation_time': 0.0036,
 'eval_accuracy': 0.8536585365853658,
 'eval_runtime': 1366.8814,
 'eval_samples_per_second': 0.78,
 'eval_steps_per_second': 0.098}

### Test

#### Test from test data

In [22]:
from sklearn.metrics import classification_report
# Predict on test set
predictions = trainer.predict(tokenized_datasets["test"])

pradictions_labels = predictions.predictions.argmax(-1)

display(classification_report(predictions.label_ids, pradictions_labels))

# wrong prediction text
wrong_prediction = [i for i, j in zip(dataset["test"]["text"], pradictions_labels) if i != j]

display("False label:")
display(f"{'Text':<50}| True Label | False Label")
display(wrong_prediction)

'              precision    recall  f1-score   support\n\n           0       0.84      0.86      0.85       533\n           1       0.86      0.84      0.85       533\n\n    accuracy                           0.85      1066\n   macro avg       0.85      0.85      0.85      1066\nweighted avg       0.85      0.85      0.85      1066\n'

'False label:'

'Text                                              | True Label | False Label'

['lovingly photographed in the manner of a golden book sprung to life , stuart little 2 manages sweetness largely without stickiness .',
 'consistently clever and suspenseful .',
 'it\'s like a " big chill " reunion of the baader-meinhof gang , only these guys are more harmless pranksters than political activists .',
 'the story gives ample opportunity for large-scale action and suspense , which director shekhar kapur supplies with tremendous skill .',
 'red dragon " never cuts corners .',
 'fresnadillo has something serious to say about the ways in which extravagant chance can distort our perspective and throw us off the path of good sense .',
 'throws in enough clever and unexpected twists to make the formula feel fresh .',
 'weighty and ponderous but every bit as filling as the treat of the title .',
 "a real audience-pleaser that will strike a chord with anyone who's ever waited in a doctor's office , emergency room , hospital bed or insurance company office .",
 'generates an enor

#### Test from sintetic data

In [25]:
import torch

text = [
    "This is a positive review.",
    "This is a negative review.",
    "This is a neutral review.",
    "I love this movie",
    "I hate this movie",
    "This movie is ok",
]

inputs = tokenizer(text, padding=True, truncation=True, return_tensors="pt")

with torch.no_grad():
    inputs = {k: v.to(classification_model.device) for k, v in inputs.items()}
    outputs = classification_model(**inputs)
    display("Class logits. Shape:", outputs.logits.shape)
    display(outputs.logits)
    display("Class predictions:")
    classes = outputs.logits.argmax(dim=1)
    display(f"{'Text':<50}| Classes")
    for t, p in zip(text, classes.cpu().detach().numpy().tolist()):
        print(f"{t:<50} | {p}")


display("predictions")
display(predictions)


'Class logits. Shape:'

torch.Size([6, 2])

tensor([[-1.5559,  1.8680],
        [ 1.7637, -1.5945],
        [ 0.7335, -0.6404],
        [-1.4913,  1.6830],
        [ 1.5876, -1.3756],
        [-1.0324,  1.0955]])

'Class predictions:'

'Text                                              | Classes'

This is a positive review.                         | 1
This is a negative review.                         | 0
This is a neutral review.                          | 0
I love this movie                                  | 1
I hate this movie                                  | 0
This movie is ok                                   | 1


'predictions'

PredictionOutput(predictions=array([[-1.0524639 ,  1.4264014 ],
       [-1.8036269 ,  2.285929  ],
       [ 0.96748286, -0.8766379 ],
       ...,
       [ 1.5207143 , -1.3890846 ],
       [ 1.5202086 , -1.436084  ],
       [ 2.0415428 , -1.8171234 ]], dtype=float32), label_ids=array([1, 1, 1, ..., 0, 0, 0]), metrics={'test_loss': 0.3741458058357239, 'test_model_preparation_time': 0.0036, 'test_accuracy': 0.849906191369606, 'test_runtime': 1335.3504, 'test_samples_per_second': 0.798, 'test_steps_per_second': 0.1})

### Analisis

Secara umum model hasil tunning:
- model memiliki akurasi 0.85 terhadap data test
- model memiliki *precision* dan *recall* yang seimbang.
- data sintetis memiliki akurasi yang bagus (peneliti menganggap data neutral disana adalah negatif)

## Text Classification 2b